# Exploratory code

The software in this file represents some of our explorations and experiments that were used to make our final analysis.

## Parsing the schema file

In [ ]:
from schema import Schema
from tabulate import tabulate
from importlib import reload

In [ ]:
schema = Schema("../data/schema.csv")
print(schema)

## Starting Spark

In [ ]:
from pyspark import SparkContext

# start spark with 1 worker thread
sc = SparkContext("local[*]")
sc.setLogLevel("ERROR")

## Analysis on machine events (Q1 & Q2)

This file describes events that occurred to the machines in the cluster, as well as their specifications (initial events).

First, let us see the schema of this file's data:

In [ ]:
print(tabulate(schema.get_table_schema('machine_events')['fields'],headers='keys'))

### Machine Event RDD

In [ ]:
machine_events_get = schema.field_getters('machine_events')
machine_events = schema.load_rdd(sc,'machine_events')
machine_events.cache()

for elem in machine_events.take(5):
	print(elem)

### Distribution of CPU usage

For the following, we only account for the machine creation events. We note that this method could count a given machine more than once, should it happen to be added more than once (and removed in-between) or modified.

Distribution of machine capacity based on their CPU power:

In [ ]:
from operator import add

cpu = machine_events_get['CPUs']
event = machine_events_get['event type']
machine_id = machine_events_get['machine ID']
time = machine_events_get['time']

cpu_usage = machine_events \
    .filter(lambda row: event(row) == 0 and cpu(row) is not None) \
    .map(lambda row: (machine_id(row),row)) \
    .reduceByKey(lambda r1,r2: r1 if time(r1)>time(r2) else r2) \
    .map(lambda row: (cpu(row[1]), 1)) \
    .reduceByKey(add) \
    .sortBy(lambda x: x[0])

for usage,nb in cpu_usage.collect():
    print(f"{nb} machines have CPU {usage}")


In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
legend = cpu_usage.map(lambda x: str(x[0]) if x[0] is not None else 'None').collect()
count = cpu_usage.sortByKey().map(lambda x: x[1]).collect()
ax.bar(legend,count)
plt.show()

### Lost Computational Power

In [ ]:
machines_history = machine_events \
    .map(lambda row: (machine_id(row),row)) \
    .aggregateByKey(None,lambda acc, e: acc+[e] if acc is not None else [e],add) \
    .map(lambda machine: machine[1]) \

for e in machines_history.take(10):
    print(tabulate(e))

In [ ]:
machines_history.filter(lambda m: sorted(m)).count() - machines_history.count()

The trace period starts at 600 seconds (aka $6 \times 10^{8}$ microseconds), to obtain the total trace time, we find the latest event time and subtracted $6 \times 10^{8}$ to it.

In [ ]:
from math import ceil

total_trace_time = time(machine_events.max(lambda row: time(row))) - 6*10**8

print(f'The total trace time is {total_trace_time} µs ≈ {ceil(total_trace_time/(10**6 * 3600))} hours ≈ {ceil(total_trace_time/(10**6 * 3600 * 24))} days')

Once we have this number, computing the total computation power of a given machine is a matter of subtracting the starting time of the machine and multiplying by the CPU capacity. We assume here that the CPU capacity of a machine never changes, and we remove the machines that have some cpu values at None to prevent weird events.

In [ ]:
def lost_and_total_power(events):
    # Functions and Variables
    status = machine_events_get['event type']
    time = machine_events_get['time']
    start = time(events[0])
    cpu = machine_events_get['CPUs'](events[0])

    # Lost time
    lost_time = 0
    for i,m in enumerate(events):
        if i==0:
            continue
        if status(m)==0 and status(events[i-1])==1:
            lost_time += time(m) - time(events[i-1])
        
    # Total time
    total_time = total_trace_time if (start < 6*10**8) else (total_trace_time - start + 6*10**8)

    # Resulting power
    return (lost_time*cpu,total_time*cpu)

def has_none_cpu(events):
    cpu = machine_events_get['CPUs']
    for m in events:
        if cpu(m) is None:
            return True
    return False

print(f'There are {machines_history.filter(has_none_cpu).count()} machines with None values in their CPUs. They will not be evaluated.')

power_lost_and_total = machines_history \
    .filter(lambda m: not has_none_cpu(m)) \
    .map(lost_and_total_power) \
    .reduce(lambda p1,p2: (p1[0]+p2[0],p1[1]+p2[1]))

print(f'During the trace time, around {(power_lost_and_total[0]/power_lost_and_total[1])*100:.4f}% of the computational power was lost.\
\nThe exact percentage obtained is {(power_lost_and_total[0]/power_lost_and_total[1])*100}% !')


## Distribution of jobs and tasks per scheduling class (Q3)

We will evaluate the distribution over **submitted** tasks and jobs. It also implies that we do not consider the distribution _over time_ but only within the _set_ of all submitted tasks and jobs.

We could also consider the evolution of the scheduling class through time, looking at the event types UPDATE_PENDING and UPDATE_RUNNING. This is a different perspective where time is a key component.

### Studying jobs

We start our analysis with jobs. They should be fewer, as there can be many tasks per job, but one job per task.

Before all, the schema for the tables we will use:

In [ ]:
print(tabulate(schema.get_table_schema('job_events')['fields'], headers='keys', maxcolwidths=40))

First step, gather the events based on the task or job they refer to. We expect such lists of events to be small enough to be handled in regular python lists.

In [ ]:
job_events = schema.load_rdd(sc,'job_events')
job_events.persist()

job_events_get = schema.field_getters('job_events')

print(tabulate(job_events.take(10)))

We are interested in the job submission event, which is when they are assigned their (initial) scheduling class.

In [ ]:
# event type 0 = SUBMIT
scheduling_events = job_events \
    .filter(lambda job_event: job_events_get['event type'](job_event) == 0)

The documentation says that it is possible for a job to be submitted more than once, when it has evicted, failed or been killed but still is runnable (up to a maximal number of trials).

Do we have any such situation ?

In [ ]:
jobs_with_several_scheduling_classes = scheduling_events \
    .map(lambda job_event: (job_events_get['job ID'](job_event), 
                            [job_events_get['scheduling class'](job_event)])) \
    .reduceByKey(lambda event1, event2: event1 + event2) \
    .filter(lambda x: len(x[1]) > 1) \
    .count()

print(f'{jobs_with_several_scheduling_classes} jobs have more than one scheduling class.')

No.

Let us then proceed with gathering the scheduling classes.

In [ ]:
job_scheduling_classes = scheduling_events \
    .map(lambda event: (job_events_get['scheduling class'](event), 1)) \
    .reduceByKey(add) \
    .collect()

for scheduling_class in job_scheduling_classes:
    print(scheduling_class)

In [ ]:
def bar_plot_question3(scheduling_classes, title):
    # TODO : make this into a proper function and put it in another file (plot.py ?)
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    scheduling_class = list(map(lambda x: x[0], scheduling_classes))
    count = list(map(lambda x: x[1], scheduling_classes))
    ax.bar(scheduling_class,count)
    plt.xlabel("Scheduling classes")
    plt.ylabel("Number of jobs")
    plt.xticks(scheduling_class)
    plt.title(title)
    plt.show()

bar_plot_question3(job_scheduling_classes, "Distribution of scheduling classes for all submitted jobs")

### Repeating the analysis for tasks

Let us now do the same analysis for all tasks, irrespective of which job they belong to.

Starting with the schema of the tasks events table

In [ ]:
task_events_get = schema.field_getters('task_events')

print(tabulate(
    schema.get_table_schema('task_events')['fields'], 
    headers='keys',
    maxcolwidths=[None, 20, None, None, 40]))

Then, we load the data and filter for submission events.

In [ ]:
task_events = schema.load_rdd(sc,'task_events').persist()

task_submit_events = task_events.filter(
    lambda task_event: task_events_get['event type'](task_event) == 0)

# task_headers = map(lambda field: field['content'], task_events_schema['fields'])
# print(tabulate(task_submit_events.take(5), headers=task_headers))
for task_event in task_submit_events.take(5):
    print(task_event)

In [ ]:
tasks_with_several_scheduling_classes = task_submit_events \
    .map(lambda event: ((task_events_get['job ID'](event), task_events_get['task index'](event)), 
                        {task_events_get['scheduling class'](event)})) \
    .reduceByKey(lambda event1, event2: event1 | event2) \
    .filter(lambda x: len(x[1]) > 1)

print(f'{tasks_with_several_scheduling_classes.count()} tasks have more than one scheduling class.')

From this, we conclude that, as with jobs, no task has ever seen its scheduling class changed after being resubmitted (due to eviction, failure, kill).

In [ ]:
task_scheduling_classes = task_submit_events \
    .map(lambda event: (task_events_get['scheduling class'](event), 1)) \
    .reduceByKey(add) \
    .collect()

for scheduling_class in task_scheduling_classes:
    print(scheduling_class)

In [ ]:
bar_plot_question3(task_scheduling_classes, "Distribution of scheduling classes for all submitted tasks")

## Relation between scheduling class and eviction of tasks (Q4)

In [ ]:
scheduling_eviction_relations = task_events \
    .filter(lambda event: task_events_get['event type'](event) == 2) \
    .map(lambda event: (task_events_get['scheduling class'](event), 1)) \
    .reduceByKey(add) \
    .sortBy(lambda x: x[0])

for relations in scheduling_eviction_relations.collect():
    print(relations)

Tasks with lower scheduling class were evicted much more than those with higher scheduling class. Note that we chose to keep all eviction events, even when a single task is evicted several times.

It matches the idea that latency-sensitive tasks, represented with higher scheduling classes, need to be interrupted (namely evicted) as little as possible, so that they can be completed soon.

## Same Job -> Same machine ? (Q5)

We here take into account every task with the same job, and for each index we determine the number of machine used.

In [ ]:
nb_machines_per_job = task_events \
    .map(lambda event: (task_events_get["job ID"](event),{task_events_get["machine ID"](event)})) \
    .reduceByKey(lambda machine1,machine2: machine1 | machine2) \
    .map(lambda job: len(job[1])) 

same_and_different_distribution = nb_machines_per_job \
    .map(lambda count: "1" if count==1 else "more than 1") \
    .countByValue() \
    .items()

for stat in same_and_different_distribution:
    print(f"{stat[1]} tasks have {stat[0]} machines.")

final = list(nb_machines_per_job.countByValue().items())
final.sort(key=lambda x:x[0])

nb_machine, count = list(zip(*final))
plt.plot(nb_machine, count)

#Solution for cumulative function from https://www.geeksforgeeks.org/python-program-to-find-cumulative-sum-of-a-list/
from itertools import accumulate
import operator
 
def cumulative_sum(input_list):
    # Use the accumulate() function to perform a cumulative sum of the elements in the list
    cumulative_sum_iter = accumulate(input_list, operator.add)
    # Convert the iterator to a list and return it
    return list(cumulative_sum_iter)

cumulative_count = cumulative_sum(count)

plt.plot(nb_machine,cumulative_count)

# plt.yscale("log")
plt.show()

As we can see, it is indeed most common to have our tasks from the same job run on 1 or 2 machines.

## Comparison of resource request and resource usage for tasks (Q6)

This topic requires to do the analysis for three different resources: CPU cores, RAM and disk space. To start with, and become acquainted with the question at hand, we start with the CPU.

### Resource requests

Tasks can specify resource requirements when SUBMITTED (0) and upon UPDATE_PENDING (7) and UPDATE_RUNNING (8) events. Do they actually update their resource requests ?

In [ ]:
resource_requests = task_events \
    .filter(lambda event: task_events_get['CPU request'](event) is not None \
        and task_events_get['memory request'](event) is not None \
        and task_events_get['disk space request'](event) is not None) \
    .map(lambda event: ((task_events_get['job ID'](event), task_events_get['task index'](event)),
        ({task_events_get['CPU request'](event)}, \
        {task_events_get['memory request'](event)}, \
        {task_events_get['disk space request'](event)}))) \
    .reduceByKey(lambda r1, r2: (r1[0] | r2[0], r1[1] | r2[1], r1[2] | r2[2]))

multiple_requests = resource_requests.filter(lambda event: len(event[1][0]) > 1 or len(event[1][1]) > 1 or len(event[1][2]) > 1)

print(f"There has been {multiple_requests.count()} tasks changing their resource requests during their lifetime over a total of {resource_requests.count()}, accounting for {multiple_requests.count() / resource_requests.count() * 100}%. Here is a sample of those:")

for req in multiple_requests.take(5):
    print(f"- task {req[0]} requested the following resources: {req[1]}")

Many tasks made more than one resource request in their lifetime.

Let us also see when the CPU request is made. Is it only done at submit time or later ?

In [ ]:
print(task_events.filter(lambda event: task_events_get['CPU request'](event) is not None) \
    .map(lambda event: ((task_events_get['job ID'](event), task_events_get['task index'](event)),
        (task_events_get['event type'](event), task_events_get['time'](event)))) \
    .reduceByKey(lambda event1, event2: event1 if event1[1] < event2[1] else event2) \
    .map(lambda entry: entry[1][0]) \
    .countByValue())

We see that the vast majority of CPU requests are made at task submission. When it is not the case, the earliest CPU request is at scheduling. To our understanding, the SCHEDULE (1) event is caused by the system, not by the user, so we choose to only keep CPU requests defined at submission.

Removing the tasks who make several different CPU requests.

In [ ]:
cpu_changing_events = task_events \
    .filter(lambda event: task_events_get['CPU request'](event) is not None) \
    .map(lambda event: ((task_events_get['job ID'](event), task_events_get['task index'](event)),
        {task_events_get['event type'](event)})) \
    .reduceByKey(lambda cpu_request1, cpu_request2: cpu_request1 | cpu_request2) \
    .cache()

for event in cpu_changing_events.map(lambda event: frozenset(event[1])).distinct().collect():
    print(event)

In [ ]:
task_events \
    .filter(lambda event: task_events_get['CPU request'](event) is not None and task_events_get['event type'](event) == 0) \
    .map(lambda event: ((task_events_get['job ID'](event), task_events_get['task index'](event)), 1)) \
    .reduceByKey(add) \
    .count()

In [ ]:
resource_requests = task_events \
    .filter(lambda event: task_events_get['CPU request'](event) is not None \
        and task_events_get['memory request'](event) is not None \
        and task_events_get['disk space request'](event) is not None \
        and task_events_get['event type'](event) == 0) \
    .map(lambda event: ((task_events_get['job ID'](event), task_events_get['task index'](event)),
        (task_events_get['CPU request'](event),task_events_get['memory request'](event),task_events_get['disk space request'](event)))) \
    .reduceByKey(lambda r1, r2: r1 if r1 == r2 else None) \
    .filter(lambda event: event[1] is not None) \
    .cache()

In [ ]:
for request in resource_requests.take(5):
    print(request)

### Resource usage table

The resource usage information is stored in its own table. We will have to learn to use it, and also to match its data with the task events.

It is to be noted that resource measurement periods might overshoot after a task is killed. We might want to remove these parts of the measurements, if they prove problematic.

The documentation for the data set also specifies how the maximum measurement data is aggregated (for sub-containers), and gives a field in the table for this information. I believe, for the sake of this work, that we can spare ourselves the extra mileage of using this field, and just consider that the measurements are accurate enough.

First thing first, loading the related data set:

In [ ]:
task_usage_get = schema.field_getters('task_usage')
task_usage = schema.load_rdd(sc,'task_usage')
task_usage.cache()

for elem in task_usage.take(5):
	print(elem)

Measurement records may be missing, not implying that a task is not running. Let us count how many such missing records there are, and how many tasks (ratio) are affected.

In [ ]:
resource_fields = [
        'start time',
        'end time',
        'CPU rate',
        'canonical memory usage',
        'local disk space usage'
    ]

def entry_converter(entry):
    return tuple(task_usage_get[field](entry) for field in resource_fields)

def aggregator(accumulator, resource_usage):
    accumulator[0] += resource_usage[1]-resource_usage[0]
    for i in range(2,len(resource_usage)):
        accumulator[i-1] += resource_usage[0]*resource_usage[i]
    return accumulator

from operator import add
def combiner(accumulator1, accumulator2):
    return list(map(add, accumulator1, accumulator2))

resources_per_task = task_usage \
    .map(lambda entry: ((task_usage_get['job ID'](entry), task_usage_get['task index'](entry)),
        entry_converter(entry))) \
    .aggregateByKey([0,]*(len(resource_fields)-1), aggregator, combiner) \
    .cache()

In [ ]:
print(f"There are {resources_per_task.count()} tasks for which we have resource consumption data.")

In [ ]:
for entry in resources_per_task.take(1):
    print(entry[0])
    # print(entry[1])
    for x in entry[1]:
        print(x)

In [ ]:
joined = resource_requests \
    .join(resources_per_task) \
    .map(lambda x: (x[1][0], *x[1][1])) \
    .sortBy(lambda x: x[0]) \
    .cache()

In [ ]:
for x in joined.take(5):
    print(x)

In `joined`, the first element is a tuple of the requests for CPU, RAM, and storage space, and the following is respectively

- the cumulated duration of the task,
- the total amount of CPU.s for the task,
- the RAM used,
- the disk storage used.

In [ ]:
cpu_req_vs_coresec = joined \
    .map(lambda x: (x[0][0],(x[2],1))) \
    .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])) \
    .map(lambda x: (x[0],x[1][0]/x[1][1]))\
    .sortByKey()\
    .collect()

mem_req_vs_coresec = joined \
    .map(lambda x: (x[0][1],(x[3],1))) \
    .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])) \
    .map(lambda x: (x[0],x[1][0]/x[1][1]))\
    .sortByKey()\
    .collect()

disk_req_vs_coresec = joined \
    .map(lambda x: (x[0][2],(x[4],1))) \
    .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])) \
    .map(lambda x: (x[0],x[1][0]/x[1][1]))\
    .sortByKey()\
    .collect()

In [ ]:
S=5

plt.scatter(*zip(*cpu_req_vs_coresec),s=S)
plt.scatter(*zip(*mem_req_vs_coresec),s=S)
plt.scatter(*zip(*disk_req_vs_coresec),s=S)
# plt.yscale("log")
plt.show()

Below, the code `.map(lambda x: (x[0],x[1][0]/x[1][1]))` performs averaging

In [ ]:
cpu_req_vs_core = joined \
    .map(lambda x: (x[0][0],(x[2]/x[1],1))) \
    .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])) \
    .map(lambda x: (x[0],x[1][0]/x[1][1]))\
    .sortByKey()\
    .collect()

mem_req_vs_core = joined \
    .map(lambda x: (x[0][1],(x[3]/x[1],1))) \
    .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])) \
    .map(lambda x: (x[0],x[1][0]/x[1][1]))\
    .sortByKey()\
    .collect()

disk_req_vs_core = joined \
    .map(lambda x: (x[0][2],(x[4]/x[1],1))) \
    .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])) \
    .map(lambda x: (x[0],x[1][0]/x[1][1]))\
    .sortByKey()\
    .collect()

In [ ]:
plt.scatter(*zip(*cpu_req_vs_core),s=S)
plt.scatter(*zip(*mem_req_vs_core),s=S)
plt.scatter(*zip(*disk_req_vs_core),s=S)
#plt.xscale("log")
plt.show()

In [ ]:
rspl = plt.subplot(2, 2, 1)
rspl.scatter(*zip(*cpu_req_vs_core),s=S)
plt.yscale("log")
plt.xscale("log")
plt.title("CPU")
rspl = plt.subplot(2, 2, 2)
rspl.scatter(*zip(*mem_req_vs_core),s=S)
plt.yscale("log")
plt.xscale("log")
plt.title("memory")
rspl = plt.subplot(2, 2, 3)
rspl.scatter(*zip(*disk_req_vs_core),s=S)
plt.yscale("log")
plt.xscale("log")
plt.title("storage")
plt.show()

These plots shows that, in general, there are no trends regarding the CPU and Disk requests vs their real usage. However, we can clearly see that there is one for the memory.

## Relation between resource usage peaks and task evictions (Q7)

We will investigate, for any given machine, whether there is a correlation  between resource consumption peaks and task evictions.

### Objective

To find a correlation, we will first construct a timeline, for each machine, of the maximum resource used, and the eviction events. Fortunately, the table Task Usage contains data on the maximum resource used withing a given time window. As such, we just need to correlate these resources with the eviction events. The documentation does not mention that tasks are migrated between machines, so we do not have to account for that.

Moreover, in the documentation we learn that the traces in our task usage data are done withing periods of 300 seconds starting at 600 seconds in the timeline. As such, to have a dataset easier to look at and perform operations on, we will form our timeline in epochs starting from 0. The first epoch 0 will therefore belongs to the time frame between 600s and 900s, epoch 1 will be between 900s and 1200s and so on.

Doing so, we find that there are 17 epochs in total for both task usage and task events.

Below are some tests showing that no task usage recording are in between epochs :

In [ ]:
overlapping_epochs = task_usage \
    .map(lambda task: ((task_usage_get['start time'](task), task_usage_get['end time'](task)), 0)) \
    .filter(lambda x: x[0][0]//300000000!=(x[0][1]-1)//300000000).count()
    # The -1 on the end time is to account for the events that ends at the end of an epoch (ex : 675s -> 900s is still withing epoch 0)
    

print(f"The number of events not inside a 300s period or epoch is {overlapping_epochs}.")


### Gathering task consumption peaks for all machines

The machine resource table contains machine IDs, so we can gather task consumption peaks for all machines on the one hand, and then task eviction events for all machines on the other hand. We can assume that no task can trigger two events at the same time, so we can ignore job ID and task index and only keep the machine ID.

We gather all maximum resource values for each machine, and their epochs. This should allow to later plot, or do some form of aggregation, for instance the count of values above a threshold.

In [ ]:
def event_to_peak_resources(event):
    return {(task_usage_get['start time'](event)-600000000)//300000000 : 
                (task_usage_get['maximum CPU rate'](event), 
                task_usage_get['maximum memory usage'](event), 
                task_usage_get['maximum disk IO time'](event))
            }

def fuse_max_dictionary(d1,d2):
    for element in d2:
        if element in d1:
            a1, a2, a3 = d1[element]
            b1, b2, b3 = d2[element]
            d1[element] = (max(a1,b1),max(a2,b2),max(a3,b3))
        else :
            d1[element] = d2[element]
    return d1

default_dict = {0:(.0,.0,.0),
                1:(.0,.0,.0),
                2:(.0,.0,.0),
                3:(.0,.0,.0),
                4:(.0,.0,.0),
                5:(.0,.0,.0),
                6:(.0,.0,.0),
                7:(.0,.0,.0),
                8:(.0,.0,.0),
                9:(.0,.0,.0),
                10:(.0,.0,.0),
                11:(.0,.0,.0),
                12:(.0,.0,.0),
                13:(.0,.0,.0),
                14:(.0,.0,.0),
                15:(.0,.0,.0),
                16:(.0,.0,.0)
                }

machines_max_usage = task_usage \
    .filter(lambda event: task_usage_get['maximum memory usage'](event) is not None and \
        task_usage_get['maximum CPU rate'](event) is not None and \
        task_usage_get['maximum disk IO time'](event) is not None \
    ) \
    .map(lambda event: 
        (task_usage_get['machine ID'](event),
        event_to_peak_resources(event))) \
    .foldByKey(default_dict,lambda measurement1, measurement2: fuse_max_dictionary(measurement1,measurement2)) \
    .map(lambda machine: (machine[0],
            ([k for k,_ in machine[1].items()],
            [v[0] for _,v in machine[1].items()],
            [v[1] for _,v in machine[1].items()],
            [v[2] for _,v in machine[1].items()])
            )
        )\
    .cache()

### Gathering task eviction events for all machines

So, we mapped machine IDs to the related resource information we have. Let us now map machine IDs to eviction events. Joining the two should give us something we can analyze.

In [ ]:
from numpy import histogram,arange

task_evict_events = task_events \
    .filter(lambda task_event: task_events_get['event type'](task_event) == 2) \
    .map(lambda event: (
            task_events_get['machine ID'](event),
            [(
                (task_events_get['time'](event)-600000000)//300000000
            )]
        )
    ) \
    .reduceByKey(lambda events1, events2: events1 + events2) \
    .map(lambda machine : (machine[0],list(histogram(machine[1],arange(18))[0]))) \
    .cache()

### Correlation Study

Now, how do we associate these two sets of values ? A simple join is enough as we need to correlate datasets with the same machine ID.

In [ ]:
correlation = machines_max_usage.join(task_evict_events).map(lambda x: (x[0], (*x[1][0],x[1][1]))).cache()

In [ ]:
for e in correlation.sortBy(lambda x: sum(x[1][4]),ascending=False).take(5):
    print(f"Table for machine #{e[0]}")
    print(tabulate(e[1]))


Now that we have our data for each machines, we need to correlate the eviction events and max resource usage for each periods. To do so we will first try to determine the correlation coefficient (also known as ***Pearson Correlation Coefficient***) for each machine and each resource usage. To do so we use the already existing *numpy* function *corrcoef*.

In [ ]:
import numpy as np

coef = correlation.map(lambda e: ( \
    [np.corrcoef(np.array(e[1][4]),np.array(e[1][1]))[1][0]],\
    [np.corrcoef(np.array(e[1][4]),np.array(e[1][2]))[1][0]],\
    [np.corrcoef(np.array(e[1][4]),np.array(e[1][3]))[1][0]] \
)).reduce(lambda a,b: (a[0]+b[0],a[1]+b[1],a[2]+b[2]))

In [ ]:
from statistics import mean,median

max_coef = tuple([max(e) for e in coef])
min_coef = tuple([min(e) for e in coef])
average_coef = tuple([mean(e) for e in coef])
median_coef = tuple([median(e) for e in coef])

print(f"Maximum coefficients found : {max_coef}\nMinimum coefficients found : {min_coef}\nAverage of all coefficients : {average_coef}\nMedian coefficients : {median_coef}")

As we can see, there are machines that seems to have correlations between the maximum resource used and the number of evictions, but the whole data (especially the mean and median) suggest that no such correlation exists.

If we don't try to look at the coefficients machine by machine, but try to find the correlation coefficient of the whole data set (while keeping X being the maximum resource used and Y the number of eviction), we obtain the following numbers :

In [ ]:
cpu = correlation.map(lambda e: (e[1][1],e[1][4])).reduce(lambda a,b: (a[0]+b[0],a[1]+b[1]))
memory = correlation.map(lambda e: (e[1][2],e[1][4])).reduce(lambda a,b: (a[0]+b[0],a[1]+b[1]))
disk = correlation.map(lambda e: (e[1][3],e[1][4])).reduce(lambda a,b: (a[0]+b[0],a[1]+b[1]))

print(np.corrcoef(np.array(cpu[0]),np.array(cpu[1]))[1][0])
print(np.corrcoef(np.array(memory[0]),np.array(memory[1]))[1][0])
print(np.corrcoef(np.array(disk[0]),np.array(disk[1]))[1][0])

As such, we can safely conclude that there are no correlation, negative of positive, between the picks of resources of a machine and its eviction events. 

Just for the sake of eye candy, here are the plots for each resource types maximum usage vs task evictions, where we can clearly see no correlations :

In [ ]:
plt.scatter(*cpu)
plt.title("Max CPU vs Evictions")
plt.xlabel("Maximum CPU usage of Epoch")
plt.ylabel("Number of evictions in Epoch")
plt.show()
plt.scatter(*memory)
plt.title("Max memory vs Evictions")
plt.xlabel("Maximum memory usage of Epoch")
plt.ylabel("Number of evictions in Epoch")
plt.show()
plt.scatter(*disk)
plt.title("Max disk vs Evictions")
plt.xlabel("Maximum disk usage of Epoch")
plt.ylabel("Number of evictions in Epoch")

## Appendix

Alternate approach with SparkSQL, allowing the use of DataFrames:
from pyspark.sql import SparkSession

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local(*]") \
    .appName("Cluster analysis") \
    .getOrCreate()

### With DataFrames

It would be nice to have the tabular methods provided by DataFrames. Fortunately, Spark offers a DataFrame API, through their Spark SQL. To [load a CSV as a DataFrame](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameReader.csv.html#pyspark.sql.DataFrameReader.csv),

In [ ]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, FloatType, IntegerType, LongType
field_to_spark_type = {
    'time': LongType,
    'machine ID': LongType,
    'event type': IntegerType,
    'platform ID': StringType,
    'CPUs': FloatType,
    'Memory': FloatType
}
machine_events_schema = StructType([
    StructField(field['content'].replace(' ', '_'), field_to_spark_type[field['content']](), field['mandatory']) for field in schemas['machine_events']['fields']
])

machine_events = spark.read \
    .format('csv') \
    .option("header","true") \
    .schema(machine_events_schema) \
    .load("../data/machine_events/part-00000-of-00001.csv")

Let us check that the schema is as we expect

In [ ]:
machine_events.printSchema()

And inspect the first few data

In [ ]:
for elem in machine_events.take(5):
	print(elem.asDict())

How much events do we have?

In [ ]:
machine_events.count(), machine_events.filter(machine_events.event_type == 1).count(),machine_events.filter(machine_events.event_type == 2).count()

In [ ]:
machine_events.filter(machine_events.event_type == 0).CPUs